**1.BUSSINESS UNDERSTANDING**

This content-based filtering movie recommendation model analyzes film characteristics (such as genres, keywords, cast, and plot summaries) to identify highly similar movies. When a user watches a film, the system will recommend other movies with similar content, based on their expressed preferences. This helps to solve the problem of 'information overload' when there are thousands of movies and users don't know where to start.

**2.DATA UNDERSTANDING**


In [75]:
import numpy as np
import pandas as pd
import difflib
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from google.colab import files
import re
import nltk
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer

In [76]:
# tải tệp lên
uploaded = files.upload()
# Thêm tham số encoding và separator để xử lý lỗi UnicodeDecodeError và ParserError
df = pd.read_csv('DataFilm_G8.csv', encoding='latin-1', sep=',')
# in những dòng đầu
print(df.head())

Saving DataFilm_G8.csv to DataFilm_G8 (2).csv
  index     budget                                    genres      id  \
0     0  237000000  Action Adventure Fantasy Science Fiction   19995   
1     1  300000000                  Adventure Fantasy Action     285   
2     2  245000000                    Action Adventure Crime  206647   
3     3  250000000               Action Crime Drama Thriller   49026   
4     4  260000000          Action Adventure Science Fiction   49529   

                                            keywords original_language  \
0  culture clash future space war space colony so...                en   
1  ocean drug abuse exotic island east india trad...                en   
2         spy based on novel secret agent sequel mi6                en   
3  dc comics crime fighter terrorist secret ident...                en   
4  based on novel mars medallion space travel pri...                en   

                             original_title  \
0                            

/tmp/ipython-input-3695509447.py:4: DtypeWarning: Columns (0,1,3,10,11,15,16,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,252,253,254,255,256,257,258,259,260,261,262,263,264,265,266,267,268,269,270

In [77]:
df.head(3)

,index,budget,genres,id,keywords,original_language,original_title,overview,popularity,release_date,...,Unnamed: 655,Unnamed: 656,Unnamed: 657,Unnamed: 658,Unnamed: 659,Unnamed: 660,Unnamed: 661,Unnamed: 662,Unnamed: 663,Unnamed: 664
0,0,237000000,Action Adventure Fantasy Science Fiction,19995,culture clash future space war space colony so...,en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437.577,12/10/2009,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1,300000000,Adventure Fantasy Action,285,ocean drug abuse exotic island east india trad...,en,Pirates of the Caribbean: At World's End,"Captain Barbossa, long believed to be dead, ha...",139.082.615,5/19/2007,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,2,245000000,Action Adventure Crime,206647,spy based on novel secret agent sequel mi6,en,Spectre,A cryptic message from Bondâs past sends him...,10.737.678.799.999.900,10/26/2015,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [78]:
# kiểm tra cột
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4813 entries, 0 to 4812
Columns: 665 entries, index to Unnamed: 664
dtypes: object(665)
memory usage: 24.4+ MB


In [79]:
# kiểm tra size (bao cột, bao hàng)
df.shape

(4813, 665)

In [80]:
desired_columns = ['original_title', 'genres', 'overview','cast', 'keywords'] # Replace with your desired column names

# Select the desired columns from the DataFrame and make an explicit copy
df_selected = df[desired_columns].copy()

# Display the first 5 rows of the new DataFrame with selected columns
display(df_selected.head())

,original_title,genres,overview,cast,keywords
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",Sam Worthington Zoe Saldana Sigourney Weaver S...,culture clash future space war space colony so...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",Johnny Depp Orlando Bloom Keira Knightley Stel...,ocean drug abuse exotic island east india trad...
2,Spectre,Action Adventure Crime,A cryptic message from Bondâs past sends him...,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,spy based on novel secret agent sequel mi6
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,Christian Bale Michael Caine Gary Oldman Anne ...,dc comics crime fighter terrorist secret ident...
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",Taylor Kitsch Lynn Collins Samantha Morton Wil...,based on novel mars medallion space travel pri...


In [81]:
# kiểm tra size (bao cột, bao hàng)
df_selected.shape

(4813, 5)

In [82]:
# kiểm tra độ missing data
df_selected.isna().sum()

,0
original_title,5
genres,33
overview,9
cast,49
keywords,417


**3.DATA PREPARATION**

In [83]:
# Fill missing values in specified columns with an empty string
columns_to_fill = ['genres', 'overview', 'cast', 'keywords']
for col in columns_to_fill:
    if col in df_selected.columns:
        df_selected[col] = df_selected[col].fillna('')

# Verify that missing values have been handled
print("Missing values after filling:")
display(df_selected.isna().sum())

Missing values after filling:


,0
original_title,5
genres,0
overview,0
cast,0
keywords,0


In [84]:
display(df_selected.head())

,original_title,genres,overview,cast,keywords
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",Sam Worthington Zoe Saldana Sigourney Weaver S...,culture clash future space war space colony so...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",Johnny Depp Orlando Bloom Keira Knightley Stel...,ocean drug abuse exotic island east india trad...
2,Spectre,Action Adventure Crime,A cryptic message from Bondâs past sends him...,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,spy based on novel secret agent sequel mi6
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,Christian Bale Michael Caine Gary Oldman Anne ...,dc comics crime fighter terrorist secret ident...
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",Taylor Kitsch Lynn Collins Samantha Morton Wil...,based on novel mars medallion space travel pri...


In [85]:
# Tải stopwords + bộ lemmatize
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [86]:
# CHUẨN BỊ STOPWORDS + LEMMATIZER
stop_words = set(stopwords.words('english'))
lemmatizer = WordNetLemmatizer()

In [87]:
# HÀM LÀM SẠCH TEXT
def clean_text(text):
    if pd.isna(text):
        return ""
# Lowercase
    text = text.lower()
# Bỏ ký tự đặc biệt, số, giữ chữ cái + khoảng trắng
    text = re.sub(r'[^a-zA-Z\s]', ' ', text)
# Tách từ
    words = text.split()
# Bỏ stopwords
    words = [word for word in words if word not in stop_words]
# Lemmatization
    words = [lemmatizer.lemmatize(word) for word in words]

    return " ".join(words)

In [88]:
# làm sạch từng cột text trong df_selected

df_selected['genres_clean'] = df_selected['genres'].apply(clean_text)
print(df_selected['genres_clean'].head())

df_selected['cast_clean'] = df_selected['cast'].apply(clean_text)
print(df_selected['cast_clean'].head())

df_selected['overview_clean'] = df_selected['overview'].apply(clean_text)
print(df_selected['overview_clean'].head())

df_selected['original_title_clean'] = df_selected['original_title'].apply(clean_text)
print(df_selected['original_title_clean'].head())

df_selected['keywords_clean'] = df_selected['keywords'].apply(clean_text)
print(df_selected['keywords_clean'].head())

0    action adventure fantasy science fiction
1                    adventure fantasy action
2                      action adventure crime
3                 action crime drama thriller
4            action adventure science fiction
Name: genres_clean, dtype: object
0    sam worthington zoe saldana sigourney weaver s...
1    johnny depp orlando bloom keira knightley stel...
2    daniel craig christoph waltz l u e seydoux ral...
3    christian bale michael caine gary oldman anne ...
4    taylor kitsch lynn collins samantha morton wil...
Name: cast_clean, dtype: object
0    nd century paraplegic marine dispatched moon p...
1    captain barbossa long believed dead come back ...
2    cryptic message bond past sends trail uncover ...
3    following death district attorney harvey dent ...
4    john carter war weary former military captain ...
Name: overview_clean, dtype: object
0                        avatar
1    pirate caribbean world end
2                       spectre
3              dark kn

In [89]:
# GHÉP 5 CỘT VÀ CHUYỂN THÀNH CỘT METADATA
df['metadata'] = (df_selected['genres_clean'] + " " + df_selected['cast_clean'] + " " + df_selected['overview_clean'] + " " + df_selected['original_title_clean'] + " " + df_selected['keywords_clean'])

# XEM KẾT QUẢ
display(df_selected.head())
display(df['metadata'].head())

,original_title,genres,overview,cast,keywords,genres_clean,cast_clean,overview_clean,original_title_clean,keywords_clean
0,Avatar,Action Adventure Fantasy Science Fiction,"In the 22nd century, a paraplegic Marine is di...",Sam Worthington Zoe Saldana Sigourney Weaver S...,culture clash future space war space colony so...,action adventure fantasy science fiction,sam worthington zoe saldana sigourney weaver s...,nd century paraplegic marine dispatched moon p...,avatar,culture clash future space war space colony so...
1,Pirates of the Caribbean: At World's End,Adventure Fantasy Action,"Captain Barbossa, long believed to be dead, ha...",Johnny Depp Orlando Bloom Keira Knightley Stel...,ocean drug abuse exotic island east india trad...,adventure fantasy action,johnny depp orlando bloom keira knightley stel...,captain barbossa long believed dead come back ...,pirate caribbean world end,ocean drug abuse exotic island east india trad...
2,Spectre,Action Adventure Crime,A cryptic message from Bondâs past sends him...,Daniel Craig Christoph Waltz L\u00e9a Seydoux ...,spy based on novel secret agent sequel mi6,action adventure crime,daniel craig christoph waltz l u e seydoux ral...,cryptic message bond past sends trail uncover ...,spectre,spy based novel secret agent sequel mi
3,The Dark Knight Rises,Action Crime Drama Thriller,Following the death of District Attorney Harve...,Christian Bale Michael Caine Gary Oldman Anne ...,dc comics crime fighter terrorist secret ident...,action crime drama thriller,christian bale michael caine gary oldman anne ...,following death district attorney harvey dent ...,dark knight rise,dc comic crime fighter terrorist secret identi...
4,John Carter,Action Adventure Science Fiction,"John Carter is a war-weary, former military ca...",Taylor Kitsch Lynn Collins Samantha Morton Wil...,based on novel mars medallion space travel pri...,action adventure science fiction,taylor kitsch lynn collins samantha morton wil...,john carter war weary former military captain ...,john carter,based novel mar medallion space travel princess


,metadata
0,action adventure fantasy science fiction sam w...
1,adventure fantasy action johnny depp orlando b...
2,action adventure crime daniel craig christoph ...
3,action crime drama thriller christian bale mic...
4,action adventure science fiction taylor kitsch...


**4.MODELING**

**A. VECTORIZATION WITH TF-IDF**

In [90]:
# Khởi tạo TfidfVectorizer
tfidf = TfidfVectorizer(stop_words='english', max_features=30000, ngram_range=(1,2), sublinear_tf=True, min_df=2, norm="l2")

# Áp dụng TF-IDF lên cột 'metadata'
tfidf_matrix = tfidf.fit_transform(df['metadata'])

# In ra kích thước của ma trận TF-IDF
print("Kích thước ma trận TF-IDF:", tfidf_matrix.shape)

Kích thước ma trận TF-IDF: (4813, 28431)


In [91]:
# ví dụ thử 1 bộ phim thành vecto
print("Kích thước ma trận TF-IDF:", tfidf_matrix.shape)

# Lấy vector TF-IDF cho bộ phim đầu tiên (index 0)
movie_0_tfidf_vector = tfidf_matrix[0].toarray().flatten()

# Lấy danh sách các từ (features) từ vectorizer
feature_names = tfidf.get_feature_names_out()

# Tạo một DataFrame để dễ đọc
tfidf_df = pd.DataFrame({
    'Feature': feature_names,
    'TF-IDF Score': movie_0_tfidf_vector
})

# Lọc và hiển thị các từ có điểm TF-IDF khác 0 cho bộ phim đầu tiên
display(tfidf_df[tfidf_df['TF-IDF Score'] > 0].sort_values(by='TF-IDF Score', ascending=False).head())

Kích thước ma trận TF-IDF: (4813, 28431)


,Feature,TF-IDF Score
10079,future space,0.181152
9575,following order,0.181152
27298,war space,0.181152
23714,space colony,0.181152
23709,space,0.178751


In [92]:
# Tính toán độ tương đồng cosine giữa các vector TF-IDF
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

# In ra kích thước của ma trận độ tương đồng
print("Kích thước ma trận độ tương đồng cosine:", cosine_sim.shape)

# Tạo DataFrame với tên phim làm chỉ mục và cột
cosine_sim_df = pd.DataFrame(cosine_sim, index=df['original_title'], columns=df['original_title'])

print("5x5 dòng đầu tiên của ma trận độ tương đồng:")
display(cosine_sim_df.head())

Kích thước ma trận độ tương đồng cosine: (4813, 4813)
5x5 dòng đầu tiên của ma trận độ tương đồng:


original_title,Avatar,Pirates of the Caribbean: At World's End,Spectre,The Dark Knight Rises,John Carter,Spider-Man 3,Tangled,Avengers: Age of Ultron,Harry Potter and the Half-Blood Prince,Batman v Superman: Dawn of Justice,...,On The Downlow,Sanctuary: Quite a Conundrum,Bang,Primer,Cavite,El Mariachi,Newlyweds,"Signed, Sealed, Delivered",Shanghai Calling,My Date with Drew
original_title,,,,,,,,,,,,,,,,,,,,,
Avatar,1.000000,0.020807,0.013514,0.012189,0.047044,0.024212,0.002115,0.038790,0.018091,0.031457,...,0.000000,0.000000,0.014503,0.034008,0.000000,0.002216,0.0,0.008020,0.016132,0.000000
Pirates of the Caribbean: At World's End,0.020807,1.000000,0.005730,0.001705,0.033383,0.039135,0.001910,0.014088,0.016342,0.022119,...,0.004722,0.020090,0.010631,0.000000,0.000000,0.008209,0.0,0.018802,0.006963,0.000000
Spectre,0.013514,0.005730,1.000000,0.015845,0.032307,0.011268,0.002204,0.043484,0.020761,0.017824,...,0.023668,0.000000,0.000000,0.000000,0.016279,0.005583,0.0,0.010706,0.009831,0.000000
The Dark Knight Rises,0.012189,0.001705,0.015845,1.000000,0.007383,0.009880,0.009029,0.026802,0.021936,0.127311,...,0.015563,0.002091,0.019738,0.015946,0.001732,0.014198,0.0,0.012990,0.024128,0.023643
John Carter,0.047044,0.033383,0.032307,0.007383,1.000000,0.078576,0.008274,0.054971,0.002789,0.023086,...,0.004763,0.000000,0.000000,0.023109,0.021617,0.002019,0.0,0.003071,0.000000,0.000000


**B. RECOMMENDATION FUNCTION (Top‑K similar movies)**

In [93]:
indices = pd.Series(df.index, index=df['original_title']).drop_duplicates()

title2idx = indices.to_dict()
idx2title = {idx: title for title, idx in title2idx.items()}

# Use the pre-computed cosine similarity matrix
sim_matrix = cosine_sim

#Recommendation function (Top‑K similar movies)
def recommend(title, top_k=5, exclude_self=True):
    if title not in title2idx:
        raise ValueError(f"Unknown title: {title}")
    idx = title2idx[title]
    scores = sim_matrix[idx].copy()

    if exclude_self:
        scores[idx] = -1.0

    # Get top-k indices by similarity
    top_indices = np.argsort(scores)[::-1][:top_k]
    results = []
    for j in top_indices:
        results.append({
            "title": idx2title[j],
            "score": float(scores[j])
        })
    return results

# Demo: Recommendations for Avengers: Age of Ultron
recs = recommend("Avengers: Age of Ultron", top_k=5)
print("\nRecommendations for Avengers: Age of Ultronr:")
for r in recs:
    print(f"  - {r['title']} (score={r['score']:.3f})")


Recommendations for Avengers: Age of Ultronr:
  - The Avengers (score=0.457)
  - Captain America: Civil War (score=0.341)
  - Iron Man 2 (score=0.281)
  - Iron Man 3 (score=0.215)
  - Iron Man (score=0.193)


**5.EVALUATION**

**RECALL**

In [94]:
from sklearn.metrics import recall_score
def recall_at_k(true_title, sim_matrix, title2idx, idx2title, k=5):
    if true_title not in title2idx or pd.isna(true_title):
        return np.nan, [], set()

    idx = title2idx[true_title]
    scores = sim_matrix[idx].copy()
    scores[idx] = -1

    # Get indices of top similar movies from the full similarity matrix
    all_rec_indices = np.argsort(scores)[::-1]

    rec_titles = []
    for j in all_rec_indices:
        if j in idx2title:
            rec_titles.append(idx2title[j])
        if len(rec_titles) == k:
            break

    target_rows = df_selected.loc[df_selected['original_title'] == true_title, 'genres']
    if target_rows.empty:
        return np.nan, [], set()

    target_movie_genres = target_rows.iloc[0]
    target_genres = set(target_movie_genres.split()) if isinstance(target_movie_genres, str) else set()

    df_temp = df_selected.dropna(subset=['original_title'])

    gt_mask_temp = df_temp['genres'].apply(lambda g: len(set(g.split()).intersection(target_genres)) > 0 if isinstance(g, str) else False)

    gt_titles = set(df_temp.loc[gt_mask_temp, 'original_title'])
    gt_titles.discard(true_title)

    hits = sum([r in gt_titles for r in rec_titles])
    recall = hits / min(len(gt_titles), k) if len(gt_titles) > 0 else 0.0
    return recall, rec_titles, gt_titles

valid_titles_df = df.dropna(subset=['original_title'])
indices = pd.Series(valid_titles_df.index, index=valid_titles_df['original_title']).drop_duplicates()
title2idx = indices.to_dict()
idx2title = {idx: title for title, idx in title2idx.items()}


# Sử dụng ma trận độ tương đồng cosine được tính toán trước
sim_matrix = cosine_sim

#Recommendation function (Top‑K similar movies)
def recommend(title, top_k=5, exclude_self=True):
    if title not in title2idx:
        raise ValueError(f"Unknown title: {title}")
    idx = title2idx[title]
    scores = sim_matrix[idx].copy()

    if exclude_self:
        scores[idx] = -1.0

    all_top_indices = np.argsort(scores)[::-1]

    results = []
    for j in all_top_indices:
        if j in idx2title:
            results.append({
                "title": idx2title[j],
                "score": float(scores[j])
            })
        if len(results) == top_k:
            break
    return results

recall_scores = []
n_movies = len(df)

print("Recall@5 for the first 5 movies:")
for i, movie in enumerate(df['original_title']):
    if pd.isna(movie):
        if i < 5:
            print(f"Movie at index {i} has NaN title, skipping evaluation.")
        continue

    if movie in title2idx:
        rec, rec_list, gt_list = recall_at_k(movie, cosine_sim, title2idx, idx2title, k=5)
        recall_scores.append(rec)
        if i < 5: # Only print for the first 5
            print(f"Movie: {movie}, Recall@5: {rec:.3f}")
    else:
        if i < 5:
            print(f"Movie: {movie} not found in title2idx, skipping evaluation.")

print(f"\nTrung bình recall@5 cho tất cả {n_movies} phim: {np.nanmean(recall_scores):.3f}")

Recall@5 for the first 5 movies:
Movie: Avatar, Recall@5: 1.000
Movie: Pirates of the Caribbean: At World's End, Recall@5: 0.800
Movie: Spectre, Recall@5: 1.000
Movie: The Dark Knight Rises, Recall@5: 1.000
Movie: John Carter, Recall@5: 0.800

Trung bình recall@5 cho tất cả 4813 phim: 0.777


**PRECISION**

In [95]:
from sklearn.metrics import precision_score

def precision_at_k(true_title, sim_matrix, title2idx, idx2title, k=5):
    if true_title not in title2idx:
        return np.nan, [], set()

    idx = title2idx[true_title]
    scores = sim_matrix[idx].copy()
    scores[idx] = -1      # Loại bỏ chính nó

    # Lấy ra tất cả các chỉ số phim được đề xuất, sắp xếp theo độ tương đồng
    all_rec_indices = np.argsort(scores)[::-1]

    rec_titles = []
    for j in all_rec_indices:
        if j in idx2title:
            rec_titles.append(idx2title[j])
        if len(rec_titles) == k:
            break

    target_movie_genres = df_selected.loc[df_selected['original_title'] == true_title, 'genres'].iloc[0]
    target_genres = set(target_movie_genres.split()) if isinstance(target_movie_genres, str) else set()

    df_temp = df_selected.dropna(subset=['original_title'])

    gt_mask = df_temp['genres'].apply(lambda g: len(set(g.split()).intersection(target_genres)) > 0 if isinstance(g, str) else False)
    gt_titles = set(df_temp.loc[gt_mask, 'original_title']) - {true_title}

    # Precision@k: tỉ lệ các phim đúng trong k phim được gợi ý
    hits = sum([r in gt_titles for r in rec_titles])
    precision = hits / k if k > 0 else np.nan
    return precision, rec_titles, gt_titles

precision_scores = []
n_movies = len(df)

print("Precision@5 for the first 5 movies:")
for i, movie in enumerate(df['original_title']):
    if pd.isna(movie):
        if i < 5:
            print(f"Movie at index {i} has NaN title, skipping evaluation.")
        continue

    if movie in title2idx:
        prec, rec_list, gt_list = precision_at_k(movie, cosine_sim, title2idx, idx2title, k=5)
        precision_scores.append(prec)
        if i < 5:
            print(f"Movie: {movie}, Precision@5: {prec:.3f}")
    else:
        if i < 5:
            print(f"Movie: {movie} not found in title2idx, skipping evaluation.")

print(f"\nTrung bình precision_k5 cho tất cả phim: {np.nanmean(precision_scores):.3f}")

Precision@5 for the first 5 movies:
Movie: Avatar, Precision@5: 1.000
Movie: Pirates of the Caribbean: At World's End, Precision@5: 0.800
Movie: Spectre, Precision@5: 1.000
Movie: The Dark Knight Rises, Precision@5: 1.000
Movie: John Carter, Precision@5: 0.800

Trung bình precision_k5 cho tất cả phim: 0.777


**RMSE**

In [96]:
from sklearn.metrics import mean_squared_error

def compute_rmse_for_genres(df_used, cosine_sim):
    y_true = []
    y_pred = []
    n_movies = len(df_used)

    all_movie_genres_sets = {}
    for i in range(n_movies):
        genres_str = df_used.loc[i, 'genres']
        if pd.isna(genres_str):
            continue
        all_movie_genres_sets[i] = set(genres_str.split())

    for i in range(n_movies):
        current_genres = all_movie_genres_sets[i]
        pred_scores_for_current_movie = cosine_sim[i]

        for j in range(n_movies):
            if i == j:
                continue

            other_genres = all_movie_genres_sets[j]

            # Determine true score (1 if common genre, 0 otherwise)
            true_score = 1 if len(current_genres & other_genres) > 0 else 0
            y_true.append(true_score)
            y_pred.append(pred_scores_for_current_movie[j])

    if not y_true:
        return np.nan

    rmse = np.sqrt(mean_squared_error(y_true, y_pred))
    return rmse

rmse_score = compute_rmse_for_genres(df_selected, cosine_sim)
print(f"Chỉ số RMSE (giữa similarity và ground truth cùng thể loại): {rmse_score:.3f}")

Chỉ số RMSE (giữa similarity và ground truth cùng thể loại): 0.694


**6.DEPLOYMENT**

The entire project source code is uploaded to GitHub


